In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Description_Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_p = html.xpath('//div[@id="pagewrap"]/p[not(@class)]')
                
                # = = = = = = = = = = = = = = =
                
                list_ul = html.xpath('//div[@id="pagewrap"]/ul')

                # = = = = = = = = = = = = = = =
                
                if len(list_p) == len(list_ul):
                    df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url']}])

                    for p, ul in zip(list_p, list_ul):
                        name = p.xpath('./*/text()')[0].strip()
                        name = name[:-1].strip() if name.endswith(':') else name
                        df_temp.loc[0, name] = '\n'.join([li.strip() for li in ul.xpath('./li/text()')])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                    # = = = = = = = = = = = = = = =
                
                    crawler_status = 'ok'
                
                    break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Description_Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./description-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./description_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：136

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/284232789435?t=1617897533000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：116] - [当前时间：15:00:05]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/302654932487?t=1560363240000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：115] - [当前时间：15:00:05]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/283631701009?t=1570069710000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：114] - [当前时间：15:00:05]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/273977223636?t=1566574565000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：113] - [当前时

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/273142831802?t=1565807691000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：82] - [当前时间：15:00:09]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/274876942277?t=1627052575000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：81] - [当前时间：15:00:09]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/303275740180?t=1617896590000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：80] - [当前时间：15:00:10]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/283601482946?t=1567396450000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：79] - [当前时间：15:00:10]



[ok] - https://vi.vipr.ebaydesc.com/itmdesc/273977062322?t=1617914496000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：4] - [剩余数量：49] - [当前时间：15:00:21]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/283589921663?t=1618334956000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：3] - [剩余数量：48] - [当前时间：15:00:21]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/273977062106?t=1617920659000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：6] - [剩余数量：47] - [当前时间：15:00:22]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/274131218964?t=1617920653000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：5] - [剩余数量：46] - [当前时间：15:00:23]



[ok] - https://vi.vipr.ebaydesc.com/itmdesc/303501946369?t=1617898144000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：5] - [剩余数量：16] - [当前时间：15:01:45]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/303379863060?t=1649704241000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：7] - [剩余数量：15] - [当前时间：15:01:46]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/275517412107?t=1666907473000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：14] - [当前时间：15:01:47]

[error] - https://vi.vipr.ebaydesc.com/itmdesc/283826055979?t=0&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：100] - [剩余数量：13] - [当前时间：15:01:48]

[ok] - 

KeyboardInterrupt
2024-03-01T07:05:44Z


KeyboardInterrupt: 